In [1]:
# Load required libraries
import numpy as np
import pandas as pd

# Custom utils from kesh-utils (Check source code: https://github.com/KeshavShetty/kesh-utils)
from KUtils.common import utils
from KUtils.eda import chartil
from KUtils.eda import data_preparation as dp
from KUtils.classifier import generic_classifier_utils as gcu

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, make_scorer, recall_score, precision_score


In [2]:
# Show all column in head()
pd.set_option('display.max_columns', None)

In [3]:
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [4]:
# Data generated from ../Sampleing.ipyb
train_inner_df = pd.read_csv('train_inner_df.csv')
train_outer_df = pd.read_csv('train_outer_df.csv')

test_inner_df = pd.read_csv('test_inner_df.csv')
test_outer_df = pd.read_csv('test_outer_df.csv')

# 1. Model 1 on inner (With Bayesian Optimization)

In [5]:
train_df = train_inner_df
# Putting feature variable to X
X = train_df.drop(['TransactionID', 'isFraud'],axis=1)

# Putting response variable to y
y = train_df['isFraud']

X_train = X
y_train = y

X_test = X
y_test = y

categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [6]:
from catboost import CatBoostClassifier, Pool, cv


cat_clf = CatBoostClassifier(learning_rate=0.01, iterations=5000, eval_metric='AUC', l2_leaf_reg=3, task_type="GPU")
cat_clf.fit(X_train, y_train, cat_features = categorical_features_indices)

ext_inner_pred_proba = cat_clf.predict_proba(test_inner_df.drop(['TransactionID'], axis=1))

0:	learn: 0.8740965	total: 63.1ms	remaining: 5m 15s
1:	learn: 0.8788894	total: 122ms	remaining: 5m 3s
2:	learn: 0.8974454	total: 181ms	remaining: 5m 1s
3:	learn: 0.9044962	total: 232ms	remaining: 4m 49s
4:	learn: 0.9071222	total: 287ms	remaining: 4m 46s
5:	learn: 0.9092288	total: 343ms	remaining: 4m 45s
6:	learn: 0.9158329	total: 397ms	remaining: 4m 42s
7:	learn: 0.9202808	total: 452ms	remaining: 4m 42s
8:	learn: 0.9213213	total: 509ms	remaining: 4m 42s
9:	learn: 0.9206242	total: 558ms	remaining: 4m 38s
10:	learn: 0.9214460	total: 613ms	remaining: 4m 38s
11:	learn: 0.9258980	total: 661ms	remaining: 4m 34s
12:	learn: 0.9277591	total: 716ms	remaining: 4m 34s
13:	learn: 0.9288279	total: 771ms	remaining: 4m 34s
14:	learn: 0.9308407	total: 820ms	remaining: 4m 32s
15:	learn: 0.9304956	total: 876ms	remaining: 4m 32s
16:	learn: 0.9334488	total: 915ms	remaining: 4m 28s
17:	learn: 0.9333581	total: 972ms	remaining: 4m 29s
18:	learn: 0.9348723	total: 1.02s	remaining: 4m 27s
19:	learn: 0.9362343	to

In [7]:
# Feature importance
importances = cat_clf.get_feature_importance()
unique_dict = dict(zip(X.dtypes.index, importances))
chartil.core_barchart_from_series(
    pd.Series(unique_dict), 
    optional_settings={'sort_by_value':True, 'decimal_precision':2}) 

# 2 Outer Model

In [5]:
train_df = train_outer_df
# Putting feature variable to X
X = train_df.drop(['TransactionID', 'isFraud'],axis=1)

# Putting response variable to y
y = train_df['isFraud']

X_train = X
y_train = y

X_test = X
y_test = y

categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [7]:
from catboost import CatBoostClassifier, Pool, cv


cat_clf = CatBoostClassifier(learning_rate=0.01, iterations=15000, eval_metric='AUC', l2_leaf_reg=3, task_type="GPU")
cat_clf.fit(X_train, y_train, cat_features = categorical_features_indices)

ext_outer_pred_proba = cat_clf.predict_proba(test_outer_df.drop(['TransactionID'], axis=1))  

0:	learn: 0.7793231	total: 68.6ms	remaining: 17m 8s
1:	learn: 0.8155191	total: 150ms	remaining: 18m 47s
2:	learn: 0.8186505	total: 228ms	remaining: 19m
3:	learn: 0.8273148	total: 292ms	remaining: 18m 12s
4:	learn: 0.8464651	total: 351ms	remaining: 17m 33s
5:	learn: 0.8454940	total: 401ms	remaining: 16m 41s
6:	learn: 0.8457619	total: 470ms	remaining: 16m 45s
7:	learn: 0.8466398	total: 539ms	remaining: 16m 50s
8:	learn: 0.8493684	total: 599ms	remaining: 16m 37s
9:	learn: 0.8553752	total: 650ms	remaining: 16m 14s
10:	learn: 0.8588194	total: 722ms	remaining: 16m 23s
11:	learn: 0.8627549	total: 781ms	remaining: 16m 15s
12:	learn: 0.8640960	total: 831ms	remaining: 15m 57s
13:	learn: 0.8689972	total: 883ms	remaining: 15m 45s
14:	learn: 0.8695523	total: 932ms	remaining: 15m 31s
15:	learn: 0.8710678	total: 983ms	remaining: 15m 20s
16:	learn: 0.8733880	total: 1.05s	remaining: 15m 22s
17:	learn: 0.8765096	total: 1.1s	remaining: 15m 14s
18:	learn: 0.8778961	total: 1.15s	remaining: 15m 5s
19:	learn

In [10]:
# Feature importance
importances = cat_clf.get_feature_importance()
unique_dict = dict(zip(X.dtypes.index, importances))
chartil.core_barchart_from_series(
    pd.Series(unique_dict), 
    optional_settings={'sort_by_value':True, 'decimal_precision':2}) 

# Apply final model on test

In [11]:
test_inner_df['isFraud'] = ext_inner_pred_proba[:,1]
test_outer_df['isFraud'] = ext_outer_pred_proba[:,1]

In [12]:
inner_tst = test_inner_df.loc[:,['TransactionID', 'isFraud' ]]
outer_tst = test_outer_df.loc[:,['TransactionID', 'isFraud' ]]

In [13]:
outer_tst.head()

,TransactionID,isFraud
0,3663549,0.001466
1,3663550,0.002985
2,3663551,0.001368
3,3663552,0.002010
4,3663553,0.004625


In [14]:
submission_csv = pd.concat([inner_tst, outer_tst])

In [15]:
submission_csv.shape

(506691, 2)

In [16]:
submission_csv.to_csv("inn_out_cat_submission.csv", index=False)
